ASAP CRN Metadata validation - wave 1

# ASAP CRN Metadata validation - wave 1

28 October 2023
Andy Henrie



## STEPS

### imports
- pandas
- pathlib

### Load CDE for validation
- check all columns


### Team Hardy
- load .csv files with tables


In [1]:
import pandas as pd
from pathlib import Path


# local helpers
from utils.qcutils import validate_table, force_enum_string, reorder_table_to_CDE
from utils.io import ReportCollector, get_dtypes_dict, read_meta_table




Streamlit NOT successfully


## Load CDE

In [2]:
CDE_path = Path.cwd() / "ASAP_CDE.csv" 
CDE = pd.read_csv(CDE_path )

CDE.head()



,Table,Field,Description,DataType,Required,Validation,Unnamed: 6,ClinPath field,team_Hafler type,ClinPath description,Unnamed: 10
0,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,NaN,NaN,NaN,NaN,bard
1,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN
2,STUDY,project_description,Project Description: Brief description of th...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN
3,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",NaN,NaN,NaN,NaN,NaN
4,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,NaN,NaN,NaN,NaN,NaN


## Clean Team Hardy tables

### Load Tables from csv

The metadata path below has copies of the raw meta-tables

In [3]:
# # AS UPLOADED FROM Team Hardy.  This is the raw meta-data
# Samples with proper batch: "Projects/ASAP/team-hardy/metadata/23102023_SAMPLE.csv"
# All other tables transferred directly to the raw bucket "Projects/ASAP/team-hardy/hardy-metadata-20232009"

# Initialize the data types dictionary
dtypes_dict = get_dtypes_dict(CDE)
    


## convert 
data_path = Path.home() / ("Projects/ASAP/team-hardy")
metadata_path = data_path / "metadata"


SUBJECT = read_meta_table(f"{metadata_path}/SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(f"{metadata_path}/CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(f"{metadata_path}/STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(f"{metadata_path}/PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(f"{metadata_path}/SAMPLE.csv", dtypes_dict)


In [10]:


# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv", dtype=dtypes_dict)
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv", dtype=dtypes_dict)
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv", dtype=dtypes_dict)
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv", dtype=dtypes_dict)
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv", dtype=dtypes_dict)

# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE)



### SUBJECT

In [11]:

SUBJECT['sex'] = SUBJECT['sex'].replace({'F':"Female", 'M':"Male"})
SUBJECT['race'] = SUBJECT['race'].replace({'W':"White", 'B':"Black or African American"})

SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({'Normal control':"Healthy Control", "Idiopathic Parkinson's disease":"Idiopathic PD"})


In [12]:
subject_report = ReportCollector(destination="print")

validate_table(SUBJECT, "SUBJECT", CDE, subject_report)

0

In [13]:
print(subject_report.get_log())

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- ethnicity: 64/64 empty rows

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in SUBJECT. 🥳



In [14]:
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,babom,P2/14,QSBB_UK,Human,Female,78,NaN,NaN,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,borah,P4/11,QSBB_UK,Human,Male,63,NaN,NaN,37.0,Idiopathic PD,clinpath info: PD | PD
2,bovon,P95/10,QSBB_UK,Human,Male,81,NaN,NaN,59.5,Idiopathic PD,clinpath info: PD | NA
3,davof,P80/11,QSBB_UK,Human,Male,80,NaN,NaN,100.0,Idiopathic PD,clinpath info: PD | PD
4,dudug,P82/10,QSBB_UK,Human,Female,87,NaN,NaN,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


### SAMPLE

`source_subject_id` are in the CLINPATH table. 

In [15]:
# SAMPLE: source_subject_id -> source_sample_id

SAMPLE[['sample_id','source_sample_id','subject_id', 'batch']]

,sample_id,source_sample_id,subject_id,batch
0,babom_ACG,,babom,2
1,babom_ACG,,babom,2
2,babom_ACG,,babom,2
3,babom_ACG,,babom,2
4,babom_ACG,,babom,2
...,...,...,...,...
3611,zupam_IPL,,zupam,5
3612,zupam_IPL,,zupam,5
3613,zupam_IPL,,zupam,5
3614,zupam_IPL,,zupam,5


In [16]:
# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# fix batch to be BATCH_1, BATCH_2, etc
SAMPLE['batch'] = "BATCH_" + SAMPLE['batch']

In [17]:
sample_report = ReportCollector(destination="print")

validate_table(SAMPLE, "SAMPLE", CDE, sample_report)

0

In [18]:
print(sample_report.get_log())

All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
	- sequencing_length:190
	>	 change to: 25, 50, 100, 150



In [19]:
SAMPLE.source_sample_id.unique()

array([''], dtype=object)

### CLINPATH

For some reason there's some duplicate 'sample_id' which also have NULL entries for some of the Enum fields.

In [20]:
CLINPATH_og = CLINPATH.copy()

CLINPATH.drop_duplicates(subset=['sample_id'], inplace=True)

# # CLINPATH.rename(columns={"subject_id":"SUBJECT_ID"}, inplace=True)
# CLINPATH['source_sample_id']


In [21]:

clinpath_report = ReportCollector(destination="print")
validate_table(CLINPATH, "CLINPATH", CDE, clinpath_report)

print(clinpath_report.get_log())


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 128/128 empty rows

	- first_motor_symptom: 128/128 empty rows

	- path_year_death: 128/128 empty rows

	- brain_weight: 128/128 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 128/128 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
	- path_autopsy_dx_main:Control brain, Pathological ageing, Control brain / Path ageing, Argyrophilic grain disease, Control brain, Cerebrovascular disease (small vessel), Cerebrovascular disease (small vessel), Control brain, Alzheimer`s disease (intermediate level AD pathological change), Control brain / Path ageing, CAA
	>	 change to: Lewy body disease nos, Parkinson's disease, Parkinson's disease with dementia, Dementia with Lewy bodies, Multiple system atrophy (SND>OPCA), Multiple system atrophy (OPCA<SND), Multiple system atrophy (SND=OPCA), Progressive supranuclear palsy, Corticobasal degeneration, Glob

In [22]:

clinpath_report2 = ReportCollector(destination="print")
validate_table(CLINPATH_og, "CLINPATH", CDE, clinpath_report2)

print(clinpath_report2.get_log())


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
	- path_autopsy_dx_main:Control brain, Pathological ageing, Control brain / Path ageing, Argyrophilic grain disease, Control brain, Cerebrovascular disease (small vessel), Cerebrovascular disease (small vessel), Control brain, Alzheimer`s disease (intermediate level AD pathological change), Control brain / Path ageing, CAA
	>	 change to: Lewy body disease nos, Parkinson's disease, Parkinson's disease with dementia, Dementia with Lewy bodies, Multiple system atrophy (SND>OPCA), Multiple system atrophy (OPCA<SND), Multiple system atrophy (SND=OPCA), Progressive supranuclea

In [23]:
# replace 'path_braak_asyn' with with string of the numeric.
# ???:  convert nan to ""?? else ""
# CLINPATH['path_braak_asyn'] = CLINPATH['path_braak_asyn'].apply(lambda val: str(int(float(val))))
CLINPATH['path_braak_asyn'].apply(lambda val: str(int(float(val)))).unique()

array(['6', '0', '5'], dtype=object)

In [24]:
CLINPATH['path_braak_nft'].unique()

array(['2', '1', '3', '0', '4', '6'], dtype=object)

In [25]:


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].replace({"0":"0", 
                                                                "1":"I", 
                                                                "2": "II", 
                                                                "3":"III", 
                                                                "4":"IV", 
                                                                "5":"V", 
                                                                "6":"VI"})


In [26]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"

# check APOE_e4_status ? currently empty
# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 
CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].replace({"0":"A0", 
                                                                    "1":"A1", 
                                                                    "2": "A2", 
                                                                    "3":"A3"})


In [27]:
CLINPATH['path_nia_aa_b'].unique()

array(['1', '2', '0', '3'], dtype=object)

In [28]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].replace({"0":"B0", 
                                                                "1":"B1", 
                                                                "2": "B2", 
                                                                "3":"B3"})



In [29]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].replace({"0":"C0", 
                                                                "1":"C1", 
                                                                "2": "C2", 
                                                                "3":"C3"})


In [30]:
CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})



In [31]:
CLINPATH[['source_sample_id','path_autopsy_dx_main','path_ad_level', "cause_death"]].drop_duplicates().head(50)


,source_sample_id,path_autopsy_dx_main,path_ad_level,cause_death
0,,Parkinson's disease with dementia,Low level Alzheimer's disease neuropathologica...,Of neurodegenerative disease_Bronchopneumonia
2,,Parkinson's disease,Low level Alzheimer's disease neuropathologica...,Ischaemia of gut
4,,Control brain,Low level Alzheimer's disease neuropathologica...,heart failure
6,,Pathological ageing,Low level Alzheimer's disease neuropathologica...,pulmonary embolism
8,,Parkinson's disease,Low level Alzheimer's disease neuropathologica...,Metastatic adenocarcinoma
10,,Parkinson's disease,No evidence of Alzheimer's disease neuropathol...,Of neurodegenerative disease
12,,Parkinson's disease with dementia,No evidence of Alzheimer's disease neuropathol...,Bronchopneumonia
14,,Parkinson's disease,Low level Alzheimer's disease neuropathologica...,Bronchopneumonia
16,,Parkinson's disease with dementia,Intermediate level Alzheimer's disease neuropa...,Of neurodegenerative disease_Urinary tract inf...
18,,Control brain,No evidence of Alzheimer's disease neuropathol...,multiorgan failure


In [32]:
CLINPATH['path_autopsy_dx_main'].unique()



array(["Parkinson's disease with dementia", "Parkinson's disease",
       'Control brain', 'Pathological ageing',
       'Control brain / Path ageing', 'Argyrophilic grain disease',
       'Control brain, Cerebrovascular disease (small vessel)',
       'Cerebrovascular disease (small vessel)',
       'Control brain, Alzheimer`s disease (intermediate level AD pathological change)',
       'Control brain / Path ageing, CAA'], dtype=object)

In [33]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [34]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)

In [35]:
CLINPATH['path_autopsy_dx_main'].unique()


array(["Parkinson's disease with dementia", "Parkinson's disease",
       'Control, no misfolded protein or significant vascular pathology',
       'Control, Argyrophilic grain disease',
       'Control, Cerebrovascular disease (atherosclerosis)',
       "Alzheimer's disease (intermediate level neuropathological change)",
       'Control, Cerebrovascular disease (cerebral amyloid angiopathy)'],
      dtype=object)

In [36]:
clinpath_report = ReportCollector(destination="print")
validate_table(CLINPATH, "CLINPATH", CDE, clinpath_report)

print(clinpath_report.get_log())

All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 128/128 empty rows

	- first_motor_symptom: 128/128 empty rows

	- path_year_death: 128/128 empty rows

	- brain_weight: 128/128 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 128/128 empty rows
## Enum fields have valid values in CLINPATH. 🥳



### STUDY

In [37]:
study_report = ReportCollector(destination="print")
validate_table(STUDY, "STUDY", CDE, study_report)


1

In [38]:
print(study_report.get_log())

All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [39]:
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date


### PROTOCOL

In [40]:
protocol_report = ReportCollector(destination="print")
validate_table(PROTOCOL, "PROTOCOL", CDE, protocol_report)


1

In [41]:

print(protocol_report.get_log())

All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



### export clean tables

In [42]:
data_path = data_path / "team-hardy"
data_path = Path.home() / ("Projects/ASAP/team-hardy")

In [54]:
data_path.name.split("-")[1]

'hardy'

In [43]:

# # write the clean metadata
# STUDY.to_csv(data_path / "metadata/STUDY.csv")
# PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
# CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
# SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
# SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hardy"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


In [44]:
# make sure cleaned files are correct

SUBJECT = read_meta_table(f"{export_root}/SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(f"{export_root}/CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(f"{export_root}/STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(f"{export_root}/PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(f"{export_root}/SAMPLE.csv", dtypes_dict)


# SUBJECT = pd.read_csv(f"{export_root}/SUBJECT.csv",header=0,index_col=0, dtype=dtypes_dict)
# CLINPATH = pd.read_csv(f"{export_root}/CLINPATH.csv",header=0,index_col=0, dtype=dtypes_dict)
# STUDY = pd.read_csv(f"{export_root}/STUDY.csv",header=0,index_col=0, dtype=dtypes_dict)
# PROTOCOL = pd.read_csv(f"{export_root}/PROTOCOL.csv",header=0,index_col=0, dtype=dtypes_dict)
# SAMPLE = pd.read_csv(f"{export_root}/SAMPLE.csv",header=0,index_col=0, dtype=dtypes_dict)


In [45]:
table, table_name = SUBJECT, "SUBJECT"

report = ReportCollector(destination="print")
validate_table(table, table_name, CDE, report)
print(report.get_log())

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- ethnicity: 64/64 empty rows

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in SUBJECT. 🥳



In [46]:
table, table_name = SAMPLE, "SAMPLE"

report = ReportCollector(destination="print")
validate_table(table, table_name, CDE, report)
print(report.get_log())

All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 3616/3616 empty rows

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
	- sequencing_length:190
	>	 change to: 25, 50, 100, 150



In [47]:
table, table_name = CLINPATH, "CLINPATH"

report = ReportCollector(destination="print")
validate_table(table, table_name, CDE, report)
print(report.get_log())

All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 128/128 empty rows

	- first_motor_symptom: 128/128 empty rows

	- path_year_death: 128/128 empty rows

	- brain_weight: 128/128 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 128/128 empty rows
## Enum fields have valid values in CLINPATH. 🥳



In [48]:
CLINPATH['path_braak_asyn'].unique()

array(['6', '0', '5'], dtype=object)

In [49]:
CLINPATH.head()

,sample_id,source_sample_id,time_from_baseline,GP2_id,hemisphere,region_level_1,region_level_2,region_level_3,AMPPD_id,family_history,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,libat_IPL,NaN,0,MDGAP-QSBB_000088_s1,Left,Parietal lobe,Inferior parietal lobule,Grey matter,NaN,Not Reported,...,NaN,A1,B1,C0,NaN,NaN,NaN,Low level Alzheimer's disease neuropathologica...,Yes,Yes
1,libat_ACG,NaN,0,MDGAP-QSBB_000088_s1,Left,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,NaN,Not Reported,...,NaN,A1,B1,C0,NaN,NaN,NaN,Low level Alzheimer's disease neuropathologica...,Yes,Yes
2,rijof_IPL,NaN,0,MDGAP-QSBB_000583_s1,Right,Parietal lobe,Inferior parietal lobule,Grey matter,NaN,Not Reported,...,NaN,A1,B1,C0,NaN,NaN,NaN,Low level Alzheimer's disease neuropathologica...,Yes,Yes
3,rijof_ACG,NaN,0,MDGAP-QSBB_000583_s1,Right,Cingulate gyrus,Anterior cingulate gyrus,Grey matter,NaN,Not Reported,...,NaN,A1,B1,C0,NaN,NaN,NaN,Low level Alzheimer's disease neuropathologica...,Yes,Yes
4,gotar_IPL,NaN,0,MDGAP-QSBB_000406_s1,Right,Parietal lobe,Inferior parietal lobule,Grey matter,NaN,Not Reported,...,NaN,A1,B1,C0,NaN,NaN,NaN,Low level Alzheimer's disease neuropathologica...,Yes,Yes


In [50]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,babom_ACG,NaN,babom,NaN,1,0,BATCH_2,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
1,babom_ACG,NaN,babom,NaN,1,0,BATCH_2,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
2,babom_ACG,NaN,babom,NaN,1,0,BATCH_2,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3,babom_ACG,NaN,babom,NaN,1,0,BATCH_2,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
4,babom_ACG,NaN,babom,NaN,1,0,BATCH_2,brain,ACG,NaN,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
